### LETTURA CARTELLA IMMAGINI

In [2]:
from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path.cwd().parents[0]  # se il notebook è in notebooks/
RAW_FLAT = PROJECT_ROOT / "data" / "raw_flat"

ALLOWED_EXT = {".jpg", ".jpeg", ".png"}

records = []
for class_dir in RAW_FLAT.iterdir():
    if not class_dir.is_dir():
        continue
    label = class_dir.name

    for img_path in class_dir.glob("*"):
        if img_path.suffix.lower() not in ALLOWED_EXT:
            continue

        records.append({
            "filepath": img_path.relative_to(PROJECT_ROOT).as_posix(),
            "label": label,
            "source": "raw_flat"
        })

df = pd.DataFrame(records)
print("Totale immagini:", len(df))
display(df.head())
print(df["label"].value_counts())


Totale immagini: 2527


,filepath,label,source
0,data/raw_flat/cardboard/cardboard1.jpg,cardboard,raw_flat
1,data/raw_flat/cardboard/cardboard10.jpg,cardboard,raw_flat
2,data/raw_flat/cardboard/cardboard100.jpg,cardboard,raw_flat
3,data/raw_flat/cardboard/cardboard101.jpg,cardboard,raw_flat
4,data/raw_flat/cardboard/cardboard102.jpg,cardboard,raw_flat


label
paper        594
glass        501
plastic      482
metal        410
cardboard    403
trash        137
Name: count, dtype: int64


### SPLIT STRATIFICATO

In [3]:
from sklearn.model_selection import train_test_split

SEED = 42
TRAIN_SIZE = 0.70
VAL_SIZE = 0.15
TEST_SIZE = 0.15

# 1) train vs temp
df_train, df_temp = train_test_split(
    df,
    test_size=(1 - TRAIN_SIZE),
    random_state=SEED,
    stratify=df["label"]
)

# 2) val vs test (split del temp)
# nel "temp" abbiamo 30%: lo separiamo in metà e metà -> 15% e 15%
df_val, df_test = train_test_split(
    df_temp,
    test_size=TEST_SIZE / (VAL_SIZE + TEST_SIZE),
    random_state=SEED,
    stratify=df_temp["label"]
)

df_train = df_train.assign(split="train")
df_val   = df_val.assign(split="val")
df_test  = df_test.assign(split="test")

df_split = pd.concat([df_train, df_val, df_test], ignore_index=True)

print(df_split["split"].value_counts())
display(pd.crosstab(df_split["split"], df_split["label"]))


split
train    1768
test      380
val       379
Name: count, dtype: int64


label,cardboard,glass,metal,paper,plastic,trash
split,,,,,,
test,60,76,62,89,73,20
train,282,350,287,416,337,96
val,61,75,61,89,72,21


In [5]:
print(df_train)

                                      filepath      label    source  split
2382       data/raw_flat/plastic/plastic92.jpg    plastic  raw_flat  train
229   data/raw_flat/cardboard/cardboard305.jpg  cardboard  raw_flat  train
352    data/raw_flat/cardboard/cardboard53.jpg  cardboard  raw_flat  train
340    data/raw_flat/cardboard/cardboard42.jpg  cardboard  raw_flat  train
951           data/raw_flat/metal/metal141.jpg      metal  raw_flat  train
...                                        ...        ...       ...    ...
2035      data/raw_flat/plastic/plastic213.jpg    plastic  raw_flat  train
709           data/raw_flat/glass/glass375.jpg      glass  raw_flat  train
2242       data/raw_flat/plastic/plastic40.jpg    plastic  raw_flat  train
610           data/raw_flat/glass/glass286.jpg      glass  raw_flat  train
974           data/raw_flat/metal/metal162.jpg      metal  raw_flat  train

[1768 rows x 4 columns]


### CONNESSIONE DB

In [15]:
import os
from dotenv import load_dotenv
import psycopg2
from psycopg2.extras import execute_values

load_dotenv()

DB_CONFIG = dict(
    host=os.getenv("DB_HOST"),
    port=int(os.getenv("DB_PORT")),
    dbname=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
)

conn = psycopg2.connect(**DB_CONFIG)
conn.autocommit = True
cur = conn.cursor()
print("✅ Connected")


✅ Connected


### INSERIMENTO NEL DB

In [ ]:
rows = df_split[["filepath", "label", "split", "source"]].to_records(index=False)

sql = """
INSERT INTO images (filepath, label, split, source)
VALUES %s
ON CONFLICT (filepath) DO UPDATE
SET label = EXCLUDED.label,
    split = EXCLUDED.split,
    source = EXCLUDED.source;
"""

execute_values(cur, sql, rows, page_size=1000)
print("✅ Insert/Upsert completato")


✅ Insert/Upsert completato


#### Verifica

In [17]:
cur.execute("SELECT split, label, COUNT(*) FROM images GROUP BY split, label ORDER BY split, label;")
cur.fetchall()[:]


[('test', 'cardboard', 60),
 ('test', 'glass', 76),
 ('test', 'metal', 62),
 ('test', 'paper', 89),
 ('test', 'plastic', 73),
 ('test', 'trash', 20),
 ('train', 'cardboard', 282),
 ('train', 'glass', 350),
 ('train', 'metal', 287),
 ('train', 'paper', 416),
 ('train', 'plastic', 337),
 ('train', 'trash', 96),
 ('val', 'cardboard', 61),
 ('val', 'glass', 75),
 ('val', 'metal', 61),
 ('val', 'paper', 89),
 ('val', 'plastic', 72),
 ('val', 'trash', 21)]

In [12]:
cur.execute("SELECT * FROM images")
cur.fetchall()[:10]


[(1,
  'data/raw_flat/plastic/plastic92.jpg',
  'plastic',
  'train',
  'raw_flat',
  None,
  None,
  None,
  None,
  datetime.datetime(2026, 1, 30, 16, 42, 54, 754164, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600)))),
 (2,
  'data/raw_flat/cardboard/cardboard305.jpg',
  'cardboard',
  'train',
  'raw_flat',
  None,
  None,
  None,
  None,
  datetime.datetime(2026, 1, 30, 16, 42, 54, 754164, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600)))),
 (3,
  'data/raw_flat/cardboard/cardboard53.jpg',
  'cardboard',
  'train',
  'raw_flat',
  None,
  None,
  None,
  None,
  datetime.datetime(2026, 1, 30, 16, 42, 54, 754164, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600)))),
 (4,
  'data/raw_flat/cardboard/cardboard42.jpg',
  'cardboard',
  'train',
  'raw_flat',
  None,
  None,
  None,
  None,
  datetime.datetime(2026, 1, 30, 16, 42, 54, 754164, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600)))),
 (5,
  'data/raw_flat/metal/metal141.jpg',
  'metal',
  '

In [13]:
cur.close()
conn.close()
